# Assignment 1: Exploring the applicability of CLIP for a cultural heritage image collection


## Due: 2025-03-17

See [Assignment 1](https://github.com/erdmann/MVA_2025/blob/main/assignments/MVA_2025_AICVCH_Assignment_1.ipynb) for the main introduction to the dataset.

This assignment applies the model presented in ["Learning Transferable Visual Models From Natural Language Supervision" by Radford et al. (arXiv, Feb. 2021)](https://arxiv.org/abs/2103.00020).

The goal of this assignment is first to use a [pre-trained CLIP model](https://github.com/openai/CLIP) to repeat and expand on assignment 1 generate semantic embedding vectors for each painting and to organize them according to cosine similarity.  Then with the resulting painting grid, we will use the multimodal embedding capability of CLIP to map regions with similarity to certain captions.


## IMPORTANT: rename your submitted .ipynb or .zip file to contain your family name somewhere prior to submission.  

Submit your .ipynb, or, if you have more than one file, a .zip file, to the "turnin" URL provided during class (redacted here because this repository is public).

## Name:
## Student ID:
## Date:

## Part 1: make a database of CLIP image embeddings

Using the largest pre-trained CLIP model you can manage, for each of the images in the Rijksmuseum paintings folder (or another large collection of cultural heritage images that you collect yourself), make use of the preprocessor that is included with the CLIP model to preprocess each image, and run the network in inference mode (no gradient accumulation) to make a database of high-dimensional semantic embeddings, one per image.  Normalize each of the embeddings so they all fall on the unit hypersphere.

Show all your code.  Indicate where you have used AI to assist you where approproate.  Comment the code, but only give comments where they are useful and non-obvious.

## Part 2: TSNE or Umap dimension reduction

Use a modern nonlinear dimension reduction algorithm such as TSNE or Umap to reduce the collection of high-dimensional embeddings to a 2D point cloud in which images with similar embeddings will appear near each other.  Use a cosine similarity metric.  Make an attractive scatter plot of the results with simple points.  Color the points according to the year of production of the artwork as extracted from the metadata JSON file. (Beware that for a few paintings, no year is provided.  Replace these with the median year of the remainder.

What do you observe about the consistency of the year as compared to the similar plot from Assignment 1?

## Part 3: Converting the point cloud to a regular grid

The number of images is such that they will fit evenly into a 68 column x 67 row grid.  Use the `linear_sum_assignment` function from scipy (`from scipy.optimize import linear_sum_assignment`) to solve the "assignment problem" of placing the images into a grid such that they closely mimic the relative arrangement of points in the point cloud from Part 2.


## Part 4: Create a single large grid image

Given the grid arrangement from Part 3, use `pyvips.arrayjoin([list of pyvips.Image], ...)` to create a single massive 68-column, 67-row image from the originals of each of the painting images arranged according to your solution from Part 3.  Save the image as a single .v (vips' very fast direct-mapping internal image format) or JPEG file.  View the image using nip2 to explore it.

Note that it is also possible to use browser-based viewing tools such as [OpenSeadragon](https://openseadragon.github.io/) to view huge images seamlessly.  Vips can be used to save an image in the DeepZoom multiresolution pyramid tile format that OSD can read.  For example, from the command line, we can run the `vips` command to convert the .v file into a DeepZoom file with the following.  Note that arguments for image encoding can be provided to vips in square brackets following the output file name.

```
$ vipsheader CLIPL14_v01_full.v
$ vips --vips-progress copy CLIPL14_v01_full.v CLIPL14_v01_full.dz[tile-size=1024,overlap=0]
vips temp-2: 54400 x 53600 pixels, 40 threads, 54400 x 1 tiles, 768 lines in buffer
vips temp-2: done in 16.8s          
$ ls | grep CLIPL14_v01_full
CLIPL14_v01_full.dzi   # metadata defining the overall size and tile size.
CLIPL14_v01_full_files # top level folder of pyramid scale subfolders containing tile jpegs
CLIPL14_v01_full.v     # original input file

```

Further note that if you would like to use the TIFF lossless file format with images greater than 4GB, they must be encoded as [BigTIFF](https://www.loc.gov/preservation/digital/formats/fdd/fdd000328.shtml) files with 64-bit internal offsets.  This can be created with vips as well, by setting the `bigtiff` option in the filename:

```
$ vips --vips-progress copy CLIPL14_v01_full.v CLIPL14_v01_full.tif[bigtiff]
```

Show your code here.


## Part 5: Neighborhood similarity

Locate the ["Still Life with Asparagus" painting](https://www.rijksmuseum.nl/en/collection/object/Still-Life-with-Asparagus--a10d24c9b6d2774e912db7afe05fb4ff) and display a 5-image x 5 image window centered on the painting.  Also do the same with your assembly from the first assignment.  Insert the images here.  Describe the differences and make a possible explanation for the relative performance of the two approaches (CNN and CLIP) on this unusual image.

## Part 6: Exploring the Joint Embedding

Using the same CLIP model, select several descriptive phrases, such as "still life with flowers", "pastoral scene with farm animals", "a civic guard portrait", "coat of arms", "a portrait of a woman", and so on, and compute the CLIP embedding for each caption.  Then make heat maps of the average cosine similarity of this embedding with the 25 nearest neighbors of each image in your grid.  (Note that this can be done in a couple lines and with no for-loops ideally.) Comment on the degree to which all of your queries result in similar levels of sharpness between high-similarity and low-similarity regions. Are there shared characteristics among "sharp" queries compared to "fuzzy" ones?

## Part 7: Exploring Semantic Boundaries

From the grid image produced in Part 4, it seems clear from both the overall pattern of colors and, on closer inspection, the local subject matter, that there are relatively homogeneous regions separated by borders of sudden transition.  Devise a strategy to attempt to map these clusters discretely, either using traditional cluster analysis such as DBSCAN or HDBSCAN or by finding areas of high variability at the borders between the regions.  Can you find archetype images in the set corresponding to each of the clusters?

Make visualizations of the 68x67 grid in which each grid cell is colored according to which of your discrete clusters it belongs to.

## Part 8: Exploring Semantic Transitions

Experiment with ways to find a sequence of images that represent a smooth transition, in the CLIP embedding sense, between a pair of distinct images.  In other words, given two "query" images in your dataset, attempt to devise a way to show a sequence of 10 to 20 intermediate images that start similar to the first image and progressively transition to high similarity with the 2nd image, thereby showing a journey in semantic space with the two query images as endpoints.

Show the code along with the results for several query pairs.

Does your algorithm rely on a highly-warping dimension reduction, such as the one used in part 2?  What are the practical consequences of this in terms of the sequences the algorithms generates?

## Part 9: (Optional) Exploring Semantic Directions

### Step 1. Make a query-by-image functionality

Using the CLIP model, write a function that will take an image as a query and return the 50 most similar images as measured by cosine similarity in the CLIP joint embedding space.

### Step 2. Mixed-mode query 
Experiment with finding "semantic directions" in the dataset using CLIP.  For example, devise a "mixed-mode" query technique that will take as input (1) a query image and (2) a text description of a semantic direction and will return images that are like the first image but semantically offset according to the text.  For example, given a landscape painting image and a semantic modifier of "snow", it would find images like the first image but with snow in them.